In [17]:
import numpy as np
import pysocialforce as psf
import random

Parameters:

In [43]:
# Number of agents
n = 3

# Define goal position
goal_position = np.array([2, 17.5])
goal_position2 = np.array([-10, 7.5])
goal_position3 = np.array([10, 7.5])

goals = np.array([goal_position, goal_position2, goal_position3])
obstacles = [[-10, 0, 15, 15], [2.5, 10, 15, 15], [-5, -5, 7.5, 15], [-5, -5, 5, 0], [5, 5, 7.5, 15], [5, 5, 5, 0], [-5, 5, 0, 0]]

# Make dictionary with the knowledge of all agents
ped_attribute_dict = {}
for i in range(n):
    n_known_goals = random.randint(0, len(goals)) # Amount of exits known to person (random)
    ped_attribute_dict[i] = {'disaster_knowledge': False, 'exit_knowledge': random.sample(list(goals), n_known_goals)}

In [46]:
# Helper function: aligns velocity with goal
def initialize_velocity(position, goal):
    direction = goal - position
    norm = np.linalg.norm(direction)
    if norm == 0:
        return np.array([0, 0])
    return direction / norm  # Unit vector towards the goal

# Initialize each agent
initial_state = np.array([])
agent = []
for i in range(n):
	goal = random.choice(goals)
	x = random.uniform(-5, 5)
	y = random.uniform(0, 15)
	vx = initialize_velocity(np.array([x, y]), goal)[0]
	vy = initialize_velocity(np.array([x, y]), goal)[1]
	agent.append([x, y, vx, vy, goal[0], goal[1]])

initial_state = np.array(agent)

# Initialize the simulator
sim = psf.Simulator(
	initial_state, groups=None, obstacles=obstacles, config_file="loes.toml"
)

# Counter for agents that have reached the goal
agents_reached_goal = 0
agents_already_counted = set()  # Set to track counted agents to avoid double counting since removing agents from the simulator is not yet working

# Function to check if an agent has reached the goal
def check_reached_goal(agent_position, y_goal=15, x_goal=5, x_goal2=-5):
	# Check if the agent has reached the goal
	if agent_position[1] > y_goal or agent_position[0] > x_goal or agent_position[0] < x_goal2:
		return True	

# Perform simulation steps and remove agents that reach the goal 
for step in range(50):
	sim.step(n=1)
	ped_states, group_states = sim.get_states()  # Access the current state of agents

	# Extract the relevant frame (assuming we need the latest frame, index 0 for instance)
	current_state = ped_states[-1]  # Use the last frame for the current state
	print(current_state[0])
	print(ped_attribute_dict[0])

	new_ped_states = []
	for i, agent_state in enumerate(current_state):
		agent_position = agent_state[:2]  # Ensure agent_position is an array of shape (2,)

		if check_reached_goal(agent_position):
			if i not in agents_already_counted:
				agents_reached_goal += 1
				agents_already_counted.add(i)
		else:
			new_ped_states.append(agent_state)
	
	# Update the simulator's state directly with agents that haven't reached the goal
	sim.ped_states = [np.array(new_ped_states)] if new_ped_states else []

	if len(new_ped_states) == 0:
		break  # Exit the loop if all agents have reached the goal

print(f"Number of agents that reached the goal: {agents_reached_goal}")

[ -3.64816018   9.66741884  -1.1598828   -0.40997772 -10.
   7.5          0.5       ]
{'disaster_knowledge': False, 'exit_knowledge': []}
[ -4.00217536   9.49988489  -0.88503795  -0.41883488 -10.
   7.5          0.5       ]
{'disaster_knowledge': False, 'exit_knowledge': []}
[ -3.80288033   9.33486132   0.49823757  -0.41255891 -10.
   7.5          0.5       ]
{'disaster_knowledge': False, 'exit_knowledge': []}
[ -3.94998669   9.18563814  -0.36776591  -0.37305795 -10.
   7.5          0.5       ]
{'disaster_knowledge': False, 'exit_knowledge': []}
[ -3.8762727    9.04429229   0.18428498  -0.35336464 -10.
   7.5          0.5       ]
{'disaster_knowledge': False, 'exit_knowledge': []}
[ -3.9330674    8.91451738  -0.14198676  -0.32443728 -10.
   7.5          0.5       ]
{'disaster_knowledge': False, 'exit_knowledge': []}
[ -3.89449677   8.79305235   0.09642659  -0.30366257 -10.
   7.5          0.5       ]
{'disaster_knowledge': False, 'exit_knowledge': []}
[ -3.92760708   8.68078068  -0.082

In [8]:
# Visualize the scene
with psf.plot.SceneVisualizer(sim, "mult_goals") as sv:
    sv.animate()

DEBUG:[pyplot.py:414       switch_backend() ] Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:[pyplot.py:414       switch_backend() ] Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:[font_manager.py:1411     _findfont_cached() ] findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:[font_manager.py:1423     _findfont_cached() ] findfont: score(FontEntry(fname='/Users/loesbijman/anaconda3/envs/my-env/lib/python3.12/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSansMono.ttf', name='DejaVu Sans Mono', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:[font_manager.py:1423     _findfont_cached() ] findfont: score(FontEntry(fname='/Users/loesbijman/anaconda3/envs/my-env/lib/python3.12/site-packages/matplotlib/mpl-data/fonts/ttf/STIXNonUniIta.ttf', name='STIXNonUnicode', style='italic', variant='normal', weight=400, stretch='norm

DEBUG:[font_manager.py:1423     _findfont_cached() ] findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/STIXGeneralBol.otf', name='STIXGeneral', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:[font_manager.py:1423     _findfont_cached() ] findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Shree714.ttc', name='Shree Devanagari 714', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:[font_manager.py:1423     _findfont_cached() ] findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/NotoSansManichaean-Regular.ttf', name='Noto Sans Manichaean', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:[font_manager.py:1423     _findfont_cached() ] findfont: score(FontEntry(fname='/System/Library/Fonts/NotoSansArmenian.ttc', name='Noto Sans Armenian', style='normal', variant='normal', weight=900, stretch='normal